In [58]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.naive_bayes import GaussianNB

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))
# chinese_stopwords = set(stopwords.words("chinese"))

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth',100)

In [6]:
train = pd.read_csv('train_2kmZucJ.csv')

In [7]:
test = pd.read_csv('test_oJQbWVk.csv')

In [8]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #iger...
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…...
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connec...
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr....
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19....


In [9]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks
1,7922,currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://insta...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is that possible?' — Yes, but wouldn't that block the ..."
3,7924,"My ipod is officially dead. I lost all my pictures and videos from the 1D and 5sos concert,and f..."
4,7925,Been fighting iTunes all night! I only want the music I $&@*# paid for


In [9]:
train.shape, train['id'].max(), test.shape#, trainTest.shape

((7920, 3), 7920, (1953, 2))

In [10]:
trainTest = pd.concat([train, test], axis = 0)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [11]:
trainTest['tweet_len'] = trainTest['tweet'].apply(lambda x: len(x))

In [149]:
trainTest[trainTest['tweet_len'] > 100].label.value_counts()

0.0    5176
1.0    1115
Name: label, dtype: int64

In [148]:
trainTest[trainTest['tweet_len'] <= 100].label.value_counts()

1.0    911
0.0    718
Name: label, dtype: int64

In [151]:
trainTest.drop(['id'], axis = 1, inplace=True)

In [152]:
trainTest.head()

,label,tweet,tweet_len
0,0.0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #iger...,128
1,0.0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…...,131
2,0.0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connec...,123
3,0.0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr....,112
4,1.0,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19....,124


In [12]:
ps = PorterStemmer()

In [14]:
lemmatizer = WordNetLemmatizer() 

In [13]:
reg_exp = re.compile('[^a-zA-Z ]+')
trainTest['cleanTweet'] = trainTest['tweet'].apply(lambda x: re.sub(reg_exp, "", x))

In [14]:
trainTest['cleanTweet'] = trainTest['cleanTweet'].apply(lambda x: " ".join([ps.stem(word).lower() for word in x.split() if word not in eng_stopwords]))

In [15]:
trainTest['cleanTweet_removed_hash'] = trainTest['cleanTweet'].apply(lambda x: x.replace("#", ""))

In [437]:
trainTest['cleanTweet'] = trainTest['cleanTweet'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split() if word not in eng_stopwords]))

In [438]:
trainTest['cleanTweet_removed_hash'] = trainTest['cleanTweet'].apply(lambda x: x.replace("#", ""))

In [466]:
trainTest.head()

,label,tweet,tweet_len,cleanTweet,cleanTweet_removed_hash
0,0.0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #iger...,128,fingerprint pregnanc test httpsgooglhmfqv android app beauti cute health iger iphoneonli iphones...,fingerprint pregnanc test httpsgooglhmfqv android app beauti cute health iger iphoneonli iphones...
1,0.0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…...,131,final transpar silicon case thank uncl yay soni xperia s sonyexperia httpinstagramcompygetjcjm,final transpar silicon case thank uncl yay soni xperia s sonyexperia httpinstagramcompygetjcjm
2,0.0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connec...,123,we love would go talk makememori unplug relax iphon smartphon wifi connect httpfbmenlsupcu,we love would go talk makememori unplug relax iphon smartphon wifi connect httpfbmenlsupcu
3,0.0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr....,112,im wire i know im georg i made way iphon cute daventri home httpinstagrampliujsk,im wire i know im georg i made way iphon cute daventri home httpinstagrampliujsk
4,1.0,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19....,124,what amaz servic appl wont even talk question i unless i pay stupid support,what amaz servic appl wont even talk question i unless i pay stupid support


In [122]:
tfidf = TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range=(1,2))

### Validation

In [213]:
TfidfVectorizer?

In [16]:
tfidf = TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range=(1,2))

In [17]:
features = tfidf.fit_transform(trainTest['cleanTweet_removed_hash'])
featuresData = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95, svd_solver = 'full')
featuresData = pca.fit_transform(featuresData)

In [18]:
featuresData.head()

,aa,aapl,ab,ab fit,abc,abc live,abercrombi,abercrombi abc,abfadoffici,abfadoffici thank,abil,abit,abl,abl get,ableton,about,about time,about to,aboutalook,aboutalook stylish,absolut,absolut best,absolut love,absurd,abt,abudhabi,abuja,abuja abujafct,abujac,abujac capitalhub,abujafct,abujafct abujapeopl,abujapeopl,abujapeopl wuse,abujaphon,abujaphon gadget,abuloui,abuloui today,abus,accept,accesorio,accesorio earphon,access,accessoir,accessoir april,accessoir free,accessoir miami,accessori,accessori accesorio,accessori bag,accessori case,accessori gadget,accessori news,accid,accident,accomplish,account,account httpcashthevisitscomindexphprefcod,accumul,ace,aceoflif,aceoflif ff,achi,achiev,acid,acquir,acquisit,across,across screen,act,act appl,act startup,action,action blog,actionpack,actionpack strategi,activ,actor,actor actress,actor shotoniphon,actorslif,actress,actress iphonesia,actress model,actual,actual cri,actual fuck,actual get,actual hate,actual work,ad,ad account,ad holidayvib,ad kid,adam,adam sandler,adapt,adaptor,add,addict,addict actionpack,addict best,addict game,addict wordphoto,addit,addit famili,address,adida,adida samsung,administr,administr complet,admit,adob,adob flash,ador,ador anim,ador babi,ador beauti,ador cute,adornthewick,adornthewick appl,adult,adult weekend,adultr,adultr pick,adultr pickup,advanc,advanc next,advantag,adventur,adventur afrolatina,adventur iphon,advertis,advertis archiv,advic,adword,adword googl,aerial,af,afford,africa,afrolatina,after,after hour,after month,after updat,after year,afternoon,again,again appl,age,agenc,agne,agneeadventur,agneeadventur india,ago,ago alreadi,ago batteri,ago charger,ago europ,ago iphon,ago samsung,ago zeeland,agre,agre appl,agre that,agricultur,agua,ah,ah appl,aha,ahead,ahg,ahg sad,ahh,ahh appl,ahhh,ahhhh,ahhhhh,ahora,ai,ai iot,aim,aint,aint fork,air,air appl,air just,air refurb,air vent,airforc,airplan,airpod,airport,aka,akg,al,al voicenot,ala,alarm,alarm doesnt,alaska,alaska kind,album,alcohol,alert,alert free,alex,alexa,alic,alienwar,aliexpress,aliexpress cheap,aliusaexpress,aliusaexpress iphon,aliv,all,all app,all appl,all black,all contact,all iphon,all music,all my,all need,all phone,all product,all set,all smile,all want,all white,all who,all wifi,allah,allday,allegedli,allen,alllllll,alllllll money,allmin,allow,allshot,allsmil,allstar,alltheway,almost,almost complet,almost dead,almost done,almost forgot,...,wwwxlradioca sikh,xadrez,xbox,xbox microsoft,xbox ps,xbox samsung,xbox soni,xboxon,xcode,xd,xd appl,xiaomi,xiaomi redmi,xiaomiindia,xlhslbccktk,xlhslbccktk love,xma,xma christma,xma present,xoxo,xp,xperia,xperia soni,xperia sp,xperia unbox,xperia xperia,xperia xperiaz,xperia xz,xperiac,xperial,xperiapicsitalia,xperiapicsitalia takenwithxperia,xperiat,xperiaz,xperiaz sexybeast,xperiaz soni,xx,xxoo,xxx,xz,xz premium,ya,ya free,ya thing,yacht,yahoo,yall,yall inkstagram,yay,yay appl,yay broke,yay christma,yay excit,yay final,yay gold,yay iphon,yay motorola,yay my,yay new,yay samsung,yay thank,yayi,yayyy,ye,ye appl,ye final,ye look,ye we,yea,yeah,yeah appl,yeah babi,yeah bitch,yeah final,yeah get,yeah io,yeah it,yeah new,yeah phone,yeah right,yeah samsung,yeah thank,yeah that,yeahstunna,yeahstunna if,year,year ago,year appl,year cheer,year christma,year cute,year get,year goal,year happi,year im,year iphon,year mari,year month,year newyear,year old,year ps,year soni,year thank,year the,year use,year virtual,year with,year year,yearold,yeay,yeeee,yellow,yellow summer,yep,yesallwomen,yesallwomen iphon,yesss,yesterday,yesterday samsung,yet,yet anoth,yet appl,yet here,yet im,yet it,yet space,yey,yey thank,yipe,ylod,yo,yoga,yogap,yogurt,yolo,yolo travel,yongnuo,yongnuo neewer,york,yosemit,yosemit appl,you,you appl,you cant,you enjoy,you find,you fuck,you get,you got,you guy,you hum,you iphon,you know,you made,you never,you pay,you should,you shouldnt,you show,you suck,you with,youd,youll,youll love,young,young instagram,young weekend,your,your appl,your bes

In [19]:
featuresData.shape, trainTest.shape

((9873, 16246), (9873, 6))

In [20]:
featuresData['tweet_len'] = trainTest['tweet_len'].reset_index(drop=True)

In [21]:
dataToTrain = featuresData.iloc[:train.shape[0],:]
dataToTest = featuresData.iloc[train.shape[0]:,:]

In [22]:
dataToTrain.shape, dataToTest.shape

((7920, 16247), (1953, 16247))

In [23]:
dataToTrain['label'] = train['label']
X_train, X_test, Y_train, Y_test = train_test_split(dataToTrain.drop(['label'], axis = 1),
                                                    dataToTrain['label'], test_size = 0.2)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
X_train.head()

,aa,aapl,ab,ab fit,abc,abc live,abercrombi,abercrombi abc,abfadoffici,abfadoffici thank,abil,abit,abl,abl get,ableton,about,about time,about to,aboutalook,aboutalook stylish,absolut,absolut best,absolut love,absurd,abt,abudhabi,abuja,abuja abujafct,abujac,abujac capitalhub,abujafct,abujafct abujapeopl,abujapeopl,abujapeopl wuse,abujaphon,abujaphon gadget,abuloui,abuloui today,abus,accept,accesorio,accesorio earphon,access,accessoir,accessoir april,accessoir free,accessoir miami,accessori,accessori accesorio,accessori bag,accessori case,accessori gadget,accessori news,accid,accident,accomplish,account,account httpcashthevisitscomindexphprefcod,accumul,ace,aceoflif,aceoflif ff,achi,achiev,acid,acquir,acquisit,across,across screen,act,act appl,act startup,action,action blog,actionpack,actionpack strategi,activ,actor,actor actress,actor shotoniphon,actorslif,actress,actress iphonesia,actress model,actual,actual cri,actual fuck,actual get,actual hate,actual work,ad,ad account,ad holidayvib,ad kid,adam,adam sandler,adapt,adaptor,add,addict,addict actionpack,addict best,addict game,addict wordphoto,addit,addit famili,address,adida,adida samsung,administr,administr complet,admit,adob,adob flash,ador,ador anim,ador babi,ador beauti,ador cute,adornthewick,adornthewick appl,adult,adult weekend,adultr,adultr pick,adultr pickup,advanc,advanc next,advantag,adventur,adventur afrolatina,adventur iphon,advertis,advertis archiv,advic,adword,adword googl,aerial,af,afford,africa,afrolatina,after,after hour,after month,after updat,after year,afternoon,again,again appl,age,agenc,agne,agneeadventur,agneeadventur india,ago,ago alreadi,ago batteri,ago charger,ago europ,ago iphon,ago samsung,ago zeeland,agre,agre appl,agre that,agricultur,agua,ah,ah appl,aha,ahead,ahg,ahg sad,ahh,ahh appl,ahhh,ahhhh,ahhhhh,ahora,ai,ai iot,aim,aint,aint fork,air,air appl,air just,air refurb,air vent,airforc,airplan,airpod,airport,aka,akg,al,al voicenot,ala,alarm,alarm doesnt,alaska,alaska kind,album,alcohol,alert,alert free,alex,alexa,alic,alienwar,aliexpress,aliexpress cheap,aliusaexpress,aliusaexpress iphon,aliv,all,all app,all appl,all black,all contact,all iphon,all music,all my,all need,all phone,all product,all set,all smile,all want,all white,all who,all wifi,allah,allday,allegedli,allen,alllllll,alllllll money,allmin,allow,allshot,allsmil,allstar,alltheway,almost,almost complet,almost dead,almost done,almost forgot,...,xadrez,xbox,xbox microsoft,xbox ps,xbox samsung,xbox soni,xboxon,xcode,xd,xd appl,xiaomi,xiaomi redmi,xiaomiindia,xlhslbccktk,xlhslbccktk love,xma,xma christma,xma present,xoxo,xp,xperia,xperia soni,xperia sp,xperia unbox,xperia xperia,xperia xperiaz,xperia xz,xperiac,xperial,xperiapicsitalia,xperiapicsitalia takenwithxperia,xperiat,xperiaz,xperiaz sexybeast,xperiaz soni,xx,xxoo,xxx,xz,xz premium,ya,ya free,ya thing,yacht,yahoo,yall,yall inkstagram,yay,yay appl,yay broke,yay christma,yay excit,yay final,yay gold,yay iphon,yay motorola,yay my,yay new,yay samsung,yay thank,yayi,yayyy,ye,ye appl,ye final,ye look,ye we,yea,yeah,yeah appl,yeah babi,yeah bitch,yeah final,yeah get,yeah io,yeah it,yeah new,yeah phone,yeah right,yeah samsung,yeah thank,yeah that,yeahstunna,yeahstunna if,year,year ago,year appl,year cheer,year christma,year cute,year get,year goal,year happi,year im,year iphon,year mari,year month,year newyear,year old,year ps,year soni,year thank,year the,year use,year virtual,year with,year year,yearold,yeay,yeeee,yellow,yellow summer,yep,yesallwomen,yesallwomen iphon,yesss,yesterday,yesterday samsung,yet,yet anoth,yet appl,yet here,yet im,yet it,yet space,yey,yey thank,yipe,ylod,yo,yoga,yogap,yogurt,yolo,yolo travel,yongnuo,yongnuo neewer,york,yosemit,yosemit appl,you,you appl,you cant,you enjoy,you find,you fuck,you get,you got,you guy,you hum,you iphon,you know,you made,you never,you pay,you should,you shouldnt,you show,you suck,you with,youd,youll,youll love,young,young instagram,young weekend,your,your appl,your best,your blowin,your

In [25]:
dataToTrain['label'].head()

0    0
1    0
2    0
3    0
4    1
Name: label, dtype: int64

In [26]:
lr = LogisticRegression(penalty = 'l1', C=2, max_iter = 100, class_weight='balanced', random_state=42)
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8939393939393939

In [30]:
from lightgbm import LGBMClassifier

In [45]:
from sklearn.metrics import f1_score
from bayes_opt import BayesianOptimization

In [56]:
def lgbROC_AUC(max_depth,n_estimators, min_data_in_leaf, learning_rate, min_sum_hessian_in_leaf, num_leaves):
#     cv = StratifiedKFold(Y, n_folds=8,shuffle=True, random_state=30)
    
    lgb1 = LGBMClassifier(max_depth=int(max_depth),n_estimators=int(n_estimators), 
                                   min_data_in_leaf=int(min_data_in_leaf), learning_rate=float(learning_rate),
                                  min_sum_hessian_in_leaf=int(min_sum_hessian_in_leaf),
                                  num_leaves=int(num_leaves))
#                                              num_leaves=int(num_leaves),reg_alpha=reg_alpha,
#                                            reg_lambda=reg_lambda)
    lgb1.fit(X_train, Y_train)
    return f1_score(Y_test, lgb1.predict(X_test), average='weighted')

rndBO = BayesianOptimization(lgbROC_AUC,{'max_depth': (30, 500),'n_estimators': (10, 5000), 'min_data_in_leaf': (20, 200),
                                        'learning_rate': (0.05,0.001), 'min_sum_hessian_in_leaf': (1,20),
                                        'num_leaves': (2,1000)})

rndBO.maximize()


|   iter    |  target   | learni... | max_depth | min_da... | min_su... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------
|  1        |  0.8327   |  0.04037  |  335.9    |  135.4    |  1.988    |  4.473e+0 |  511.8    |
|  2        |  0.849    |  0.01422  |  219.6    |  24.61    |  16.88    |  3.073e+0 |  123.0    |
|  3        |  0.8535   |  0.03081  |  320.1    |  99.34    |  11.05    |  1.444e+0 |  465.8    |
|  4        |  0.8343   |  0.03345  |  55.96    |  178.6    |  9.865    |  1.382e+0 |  678.3    |
|  5        |  0.8067   |  0.003038 |  477.9    |  158.0    |  8.632    |  613.1    |  710.3    |
|  6        |  0.8647   |  0.05     |  484.2    |  34.81    |  1.827    |  2.987e+0 |  987.2    |
|  7        |  0.8157   |  0.05     |  497.1    |  195.9    |  18.28    |  2.519e+0 |  21.55    |


c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


|  8        |  0.6208   |  0.05     |  43.11    |  52.56    |  18.98    |  14.51    |  3.083    |
|  9        |  0.8529   |  0.05     |  51.36    |  22.7     |  13.6     |  4.992e+0 |  10.44    |
|  10       |  0.835    |  0.05     |  38.26    |  23.1     |  12.83    |  4.981e+0 |  948.9    |
|  11       |  0.8358   |  0.05     |  451.4    |  34.93    |  19.8     |  1.554e+0 |  999.2    |
|  12       |  0.8151   |  0.05     |  30.39    |  197.8    |  1.655    |  3.571e+0 |  966.9    |
|  13       |  0.8485   |  0.05     |  499.9    |  45.88    |  6.887    |  4.988e+0 |  65.95    |
|  14       |  0.8375   |  0.05     |  476.1    |  20.68    |  10.15    |  3.952e+0 |  895.3    |
|  15       |  0.8628   |  0.05     |  33.27    |  27.69    |  3.628    |  2.323e+0 |  522.9    |
|  16       |  0.8536   |  0.05     |  490.5    |  29.4     |  4.666    |  2.787e+0 |  569.8    |
|  17       |  0.8198   |  0.05     |  464.5    |  172.9    |  19.86    |  4.991e+0 |  976.5    |
|  18       |  0.871

In [ ]:
|  0.8825   |  0.1      |  35.55    |  22.76    |  1.946    |  65.33    |  965.8    |

In [57]:
params = {'min_data_in_leaf': 35, 'learning_rate': 0.01705919212825226, 'metric': 'f1_score', "feature_fraction":0.8,
 'subsample': 0.9920028820065904, 'bagging_freq': 11, 'importance_type': 'gain', "bagging_fraction": 0.8, "lambda": 0.1,
 'min_sum_hessian_in_leaf': 1, 'verbose': 1, 'boost_from_average': False,
 'colsample_bytree': 0.13358144268185645, 'objective': 'binary', 
 'max_depth': 100, 'num_leaves': 17, 'verbose_eval': True}

# 'no_of_adults','age_range',
#                             'no_of_children','income_bracket'

lgb_clf1 = LGBMClassifier(max_depth=100, learning_rate=0.01, n_estimators = 2000, nthread = 4, n_jobs = -1,
                          feature_fraction=0.8)
lgb_clf1.fit(X_train, Y_train)

lgb_clf1.score(X_test, Y_test)

0.88510101010101

In [50]:
f1_score(Y_test, lgb_clf1.predict(X_test), average='weighted')

0.885281430249644

In [ ]:
0.8819444444444444  0.8863636363636364

In [48]:
print(classification_report(Y_test, lgb_clf1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1162
           1       0.80      0.76      0.78       422

    accuracy                           0.89      1584
   macro avg       0.86      0.85      0.85      1584
weighted avg       0.88      0.89      0.89      1584



In [35]:
print(classification_report(Y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93      1162
           1       0.77      0.86      0.81       422

    accuracy                           0.89      1584
   macro avg       0.86      0.88      0.87      1584
weighted avg       0.90      0.89      0.90      1584



In [418]:
GaussianNB?

In [416]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
gnb.score(X_test, Y_test)

0.8497474747474747

In [419]:
gnb

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
0.8869949494949495 0.8882575757575758

In [ ]:
Lemmatizer  0.8964646464646465(3)  0.8983585858585859(2)

In [ ]:
PS  0.8914141414141414(3) 

In [ ]:
Stopwords  0.8869949494949495 0.8914141414141414(3)

In [ ]:
Normal  0.8863636363636364 0.8939393939393939 0.9027777777777778 0.9040404040404041(4)

In [28]:
print(classification_report(Y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93      1162
           1       0.77      0.86      0.81       422

    accuracy                           0.89      1584
   macro avg       0.86      0.88      0.87      1584
weighted avg       0.90      0.89      0.90      1584



In [417]:
print(classification_report(Y_test, gnb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1148
           1       0.72      0.74      0.73       436

    accuracy                           0.85      1584
   macro avg       0.81      0.82      0.81      1584
weighted avg       0.85      0.85      0.85      1584



### Full Data

In [456]:
dataToTrain = featuresData.iloc[:train.shape[0],:]
dataToTest = featuresData.iloc[train.shape[0]:,:]

dataToTrain['label'] = train['label']

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [457]:
lr = LogisticRegression(penalty = 'l1', C=2, max_iter = 100, class_weight='balanced', random_state=42)
lr.fit(dataToTrain.drop(['label'], axis = 1), dataToTrain['label'])
# lr.score(X_test, Y_test)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=2, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [458]:
preds = lr.predict(dataToTest.drop(['label'],axis = 1))

In [459]:
preds.sum()

610

In [460]:
dataToTest['label'] = preds
dataToTest.reset_index(inplace = True)
dataToTest['id']=test['id']
dataToTest[['id','label']].head() #.reset_index(inplace = True)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1


In [461]:

dataToTest[['id', 'label']].to_csv('out11.csv',index = False)

### Finish

In [462]:
trainTest[train.shape[0]:].head()

,label,tweet,tweet_len,cleanTweet,cleanTweet_removed_hash
0,NaN,I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks,77,I hate new iphone upgrade Wont let download apps ugh apple suck,I hate new iphone upgrade Wont let download apps ugh apple suck
1,NaN,currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://insta...,115,currently shitting fucking pant apple iMac cashmoney raddest swagswagswag httpinstagrampUUISbIBZo,currently shitting fucking pant apple iMac cashmoney raddest swagswagswag httpinstagrampUUISbIBZo
2,NaN,"I'd like to puts some CD-ROMS on my iPad, is that possible?' — Yes, but wouldn't that block the ...",104,Id like put CDROMS iPad possible Yes wouldnt block screen,Id like put CDROMS iPad possible Yes wouldnt block screen
3,NaN,"My ipod is officially dead. I lost all my pictures and videos from the 1D and 5sos concert,and f...",129,My ipod officially dead I lost picture video D so concertand Vet Camp hatinglife sobbing,My ipod officially dead I lost picture video D so concertand Vet Camp hatinglife sobbing
4,NaN,Been fighting iTunes all night! I only want the music I $&@*# paid for,70,Been fighting iTunes night I want music I paid,Been fighting iTunes night I want music I paid


In [124]:
features = tfidf.fit_transform(trainTest['tweet'])

In [50]:
tfidf.get_feature_names()

['00',
 '00 http',
 '000',
 '000 pictures',
 '000 steps',
 '0051',
 '0051 beaboutique',
 '00am',
 '00pm',
 '01',
 '01 2018',
 '010111',
 '010111 virtual',
 '01108nartl',
 '01108nartl 64gb',
 '01am',
 '01ammay',
 '01pm',
 '01pmjune',
 '01pmjune 05',
 '02',
 '02 00am',
 '02 01',
 '02 06',
 '02 2017',
 '02 30am',
 '02 30pm',
 '03',
 '03 2016',
 '03 2017',
 '03 30am',
 '04',
 '04 30pm',
 '04 apple',
 '05',
 '05 2011',
 '06',
 '06 2015',
 '06 21',
 '07',
 '07 00',
 '07 00am',
 '07 01',
 '07 01am',
 '07 2015',
 '07 apple',
 '08',
 '08 00pm',
 '08 01pm',
 '08 2011',
 '08 2017',
 '08 30pm',
 '08184820028',
 '08184820028 to',
 '09',
 '09 2017',
 '0d9dzq0cxo',
 '10',
 '10 000',
 '10 2017',
 '10 30am',
 '10 and',
 '10 clock',
 '10 days',
 '10 dual',
 '10 free',
 '10 fucking',
 '10 in',
 '10 is',
 '10 min',
 '10 minutes',
 '10 month',
 '10 off',
 '10 on',
 '10 pic',
 '10 screen',
 '10 update',
 '10 years',
 '100',
 '100 an',
 '100 to',
 '100 usd',
 '1000',
 '1000 followers',
 '100happydays',
 '101

In [125]:
featuresData = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

In [126]:
featuresData.shape

(9873, 23358)

In [127]:
dataToTrain = featuresData.iloc[:train.shape[0],:]

In [128]:
dataToTrain['label'] = train['label']

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [129]:
dataToTrain = featuresData.iloc[:train.shape[0],:]
dataToTest = featuresData.iloc[train.shape[0]:,:]

In [79]:
dataToTrain.head()

,00,00 http,000,000 pictures,000 steps,0051,0051 beaboutique,00am,00pm,01,01 2018,010111,010111 virtual,01108nartl,01108nartl 64gb,01am,01ammay,01pm,01pmjune,01pmjune 05,02,02 00am,02 01,02 06,02 2017,02 30am,02 30pm,03,03 2016,03 2017,03 30am,04,04 30pm,04 apple,05,05 2011,06,06 2015,06 21,07,07 00,07 00am,07 01,07 01am,07 2015,07 apple,08,08 00pm,08 01pm,08 2011,08 2017,08 30pm,08184820028,08184820028 to,09,09 2017,0d9dzq0cxo,10,10 000,10 2017,10 30am,10 and,10 clock,10 days,10 dual,10 free,10 fucking,10 in,10 is,10 min,10 minutes,10 month,10 off,10 on,10 pic,10 screen,10 update,10 years,100,100 an,100 to,100 usd,1000,1000 followers,100happydays,101,106,106finalact,1080p,10likes,10likes apple,10pcs,10pcs exquisite,10th,11,11 11,11 2017,11 23,11 and,11 the,11 to,1111,1111 010111,112600020803,112600020803 etsy,114,12,12 2017,120,1200,128gb,12mglt,12mglt http,12th,12yrs,12yrs in,13,13 2016,13 days,1321252303,1321252303 sr,13579_3,13eatsmith,13eatsmith love,13mp,13mp camera,13th,14,140,140 to,15,15 2015,15 off,150,1500,15th,16,16 2015,16 2017,16 2018,16gb,16gb black,16gb factory,16gb unlocked,16mp,16mp camera,17,17 2017,18,18 2016,18 2017,184,184 95,19,19 2016,19 69,19 99,1995,1995 sony,1996,19thnov,19thnov http,1avvqn7jfp,1c11,1c11 china,1d,1d3dzux,1d3dzux android,1gb,1nutukg,1nutukg iphone,1pc,1pc auto,1pic,1pic twitter,1st,1st anniversary,1st generation,1wbrj1b,1wbrj1b arianasurprise,1week,20,20 2017,20 battery,20 for,20 http,20 linkcode,20 mins,20 minutes,20 off,20 via,20 years,200,200 to,2000,2000 pics,2001,2009,2009 http,2010,2010 http,2011,2011 10,2011 at,2011 iphone,2012,2012 06,2012 09,2012 11,2012 http,2012 iphone,2012 samsung,2013,2013 http,2014,2014 02,2014 at,2014 cali,2014 cheerful,2014 new,2015,2015 at,2015 http,2015 taken,2016,2016 11,2016 at,2016 https,2016 new,2016 sony,2016 taken,2017,2017 09,2017 11,2017 at,2017 last,2018,2018 01,2018 02,2018 03,2018 04,2018 06,2018 07,2018 08,2018 09,2018 10,2018 at,2018 cute,2018 metroman,2018 newyear,...,you order,you papa,you pay,you phone,you pissed,you put,you quotes,you re,you ready,you really,you samsung,you say,you scored,you see,you send,you should,you shouldn,you showed,you siri,you smile,you so,you sony,you soon,you speak,you still,you stupid,you suck,you take,you tell,you text,you the,you think,you to,you too,you try,you unlock,you update,you use,you ve,you very,you waiting,you wanna,you want,you will,you with,you won,you words,you would,you you,you your,young,young instagramers,young weekend,your,your all,your apple,your back,your brain,your business,your busy,your chargers,your computer,your contacts,your crap,your customer,your day,your device,your email,your entire,your face,your facebook,your family,your favorite,your favourite,your finger,your friends,your fucking,your galaxy,your gear,your great,your happiness,your headphones,your here,your ipad,your iphone,your ipod,your itunes,your last,your latest,your life,your love,your mind,your mistake,your mobile,your moby,your monitoring,your name,your nearest,your new,your opinion,your os,your own,your personal,your phone,your phones,your pics,your pocket,your product,your products,your questions,your screen,your search,your servers,your shitty,your smartphone,your software,your support,your time,your update,your voice,your work,your world,youre,yours,yourself,yourself and,yourself apple,yourselves,yourtechcompany,yourtechcompany pic,yousaidthat,yousaidthat http,yousuck,yousuck worst,youtu,youtu be,youtube,youtube app,youtube com,youtube facebook,youtube google,youtube https,youtube instagood,youtube music,youtube sidibouzid,youtube subscribe,youtube youtuber,youtubepic,youtubepic twitter,youtuber,youtuber photo,yr,yr old,yrs,yrs the,ysl,yum,yum food,yum goodlife,yum http,yum tasty,yum yummy,yummy,yummy chocolate,yummy delicious,yup,yup that,z1,z1 sony,z1 xperia,z10,z2,z2 unboxing,z3,z3 sony,za,zara,zaschiti,zazzle,zazzle com,zd,zebra,zebra love,zeeland,zeeland brouwersdam,zeeland dezeekust,zeeland noordzee,zeela

In [77]:
dataToTest['label'].head()

7920    0.0
7921    0.0
7922    0.0
7923    0.0
7924    0.0
Name: label, dtype: float64

In [73]:
dataToTrain.shape, dataToTest.shape

((7920, 23358), (1953, 23358))

In [21]:
from sklearn.linear_model import LogisticRegression

In [78]:
featuresData.head()

,00,00 http,000,000 pictures,000 steps,0051,0051 beaboutique,00am,00pm,01,01 2018,010111,010111 virtual,01108nartl,01108nartl 64gb,01am,01ammay,01pm,01pmjune,01pmjune 05,02,02 00am,02 01,02 06,02 2017,02 30am,02 30pm,03,03 2016,03 2017,03 30am,04,04 30pm,04 apple,05,05 2011,06,06 2015,06 21,07,07 00,07 00am,07 01,07 01am,07 2015,07 apple,08,08 00pm,08 01pm,08 2011,08 2017,08 30pm,08184820028,08184820028 to,09,09 2017,0d9dzq0cxo,10,10 000,10 2017,10 30am,10 and,10 clock,10 days,10 dual,10 free,10 fucking,10 in,10 is,10 min,10 minutes,10 month,10 off,10 on,10 pic,10 screen,10 update,10 years,100,100 an,100 to,100 usd,1000,1000 followers,100happydays,101,106,106finalact,1080p,10likes,10likes apple,10pcs,10pcs exquisite,10th,11,11 11,11 2017,11 23,11 and,11 the,11 to,1111,1111 010111,112600020803,112600020803 etsy,114,12,12 2017,120,1200,128gb,12mglt,12mglt http,12th,12yrs,12yrs in,13,13 2016,13 days,1321252303,1321252303 sr,13579_3,13eatsmith,13eatsmith love,13mp,13mp camera,13th,14,140,140 to,15,15 2015,15 off,150,1500,15th,16,16 2015,16 2017,16 2018,16gb,16gb black,16gb factory,16gb unlocked,16mp,16mp camera,17,17 2017,18,18 2016,18 2017,184,184 95,19,19 2016,19 69,19 99,1995,1995 sony,1996,19thnov,19thnov http,1avvqn7jfp,1c11,1c11 china,1d,1d3dzux,1d3dzux android,1gb,1nutukg,1nutukg iphone,1pc,1pc auto,1pic,1pic twitter,1st,1st anniversary,1st generation,1wbrj1b,1wbrj1b arianasurprise,1week,20,20 2017,20 battery,20 for,20 http,20 linkcode,20 mins,20 minutes,20 off,20 via,20 years,200,200 to,2000,2000 pics,2001,2009,2009 http,2010,2010 http,2011,2011 10,2011 at,2011 iphone,2012,2012 06,2012 09,2012 11,2012 http,2012 iphone,2012 samsung,2013,2013 http,2014,2014 02,2014 at,2014 cali,2014 cheerful,2014 new,2015,2015 at,2015 http,2015 taken,2016,2016 11,2016 at,2016 https,2016 new,2016 sony,2016 taken,2017,2017 09,2017 11,2017 at,2017 last,2018,2018 01,2018 02,2018 03,2018 04,2018 06,2018 07,2018 08,2018 09,2018 10,2018 at,2018 cute,2018 metroman,2018 newyear,...,you order,you papa,you pay,you phone,you pissed,you put,you quotes,you re,you ready,you really,you samsung,you say,you scored,you see,you send,you should,you shouldn,you showed,you siri,you smile,you so,you sony,you soon,you speak,you still,you stupid,you suck,you take,you tell,you text,you the,you think,you to,you too,you try,you unlock,you update,you use,you ve,you very,you waiting,you wanna,you want,you will,you with,you won,you words,you would,you you,you your,young,young instagramers,young weekend,your,your all,your apple,your back,your brain,your business,your busy,your chargers,your computer,your contacts,your crap,your customer,your day,your device,your email,your entire,your face,your facebook,your family,your favorite,your favourite,your finger,your friends,your fucking,your galaxy,your gear,your great,your happiness,your headphones,your here,your ipad,your iphone,your ipod,your itunes,your last,your latest,your life,your love,your mind,your mistake,your mobile,your moby,your monitoring,your name,your nearest,your new,your opinion,your os,your own,your personal,your phone,your phones,your pics,your pocket,your product,your products,your questions,your screen,your search,your servers,your shitty,your smartphone,your software,your support,your time,your update,your voice,your work,your world,youre,yours,yourself,yourself and,yourself apple,yourselves,yourtechcompany,yourtechcompany pic,yousaidthat,yousaidthat http,yousuck,yousuck worst,youtu,youtu be,youtube,youtube app,youtube com,youtube facebook,youtube google,youtube https,youtube instagood,youtube music,youtube sidibouzid,youtube subscribe,youtube youtuber,youtubepic,youtubepic twitter,youtuber,youtuber photo,yr,yr old,yrs,yrs the,ysl,yum,yum food,yum goodlife,yum http,yum tasty,yum yummy,yummy,yummy chocolate,yummy delicious,yup,yup that,z1,z1 sony,z1 xperia,z10,z2,z2 unboxing,z3,z3 sony,za,zara,zaschiti,zazzle,zazzle com,zd,zebra,zebra love,zeeland,zeeland brouwersdam,zeeland dezeekust,zeeland noordzee,zeela

In [33]:
lr = LogisticRegression?

In [34]:
train['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split

In [110]:
X_train, X_test, Y_train, Y_test = train_test_split(featuresData.drop(['label'], axis = 1),featuresData['label'], test_size = 0.2)

KeyError: "['label'] not found in axis"

In [37]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty = 'l1', C=1.0, max_iter = 100, class_weight='balanced')
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

from sklearn.metrics import classification_report
print(classification_report(Y_test, lr.predict(X_test)))

In [40]:
from sklearn.metrics import classification_report

In [43]:
print(classification_report(Y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.86      0.91      1169
           1       0.70      0.95      0.81       415

    accuracy                           0.88      1584
   macro avg       0.84      0.90      0.86      1584
weighted avg       0.91      0.88      0.89      1584



In [111]:
# Full Data
lr.fit(dataToTrain.drop(['label'], axis = 1), dataToTrain['label'])


c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
preds = lr.predict(dataToTest)

In [116]:
dataToTest['label'] = preds
dataToTest.reset_index(inplace = True)
dataToTest['id']=test['id']
dataToTest[['id','label']].head() #.reset_index(inplace = True)

In [118]:
dataToTest.shape

(1953, 26107)

In [119]:
dataToTest[['id','label']].head() #.reset_index(inplace = True)

,id,label
0,7921,1
1,7922,0
2,7923,1
3,7924,1
4,7925,1


In [120]:
dataToTest['label'].sum()

770

In [121]:
dataToTest[['id', 'label']].to_csv('out.csv',index = False)

In [131]:
from sklearn.svm import SVC

In [133]:
svm_clf = SVC()

In [134]:
svm_clf.fit(dataToTrain.drop(['label'], axis = 1), dataToTrain['label'])

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [138]:
preds = svm_clf.predict(dataToTest.drop(['label'], axis = 1))

In [141]:
preds.shape

(1953,)

In [142]:
preds.sum()

0

In [ ]:
dataToTest['label'] = preds
dataToTest.reset_index(inplace = True)
dataToTest['id']=test['id']
dataToTest[['id','label']].head() #.reset_index(inplace = True)

In [ ]:
dataToTest[['id', 'label']].to_csv('out.csv',index = False)

In [143]:
from sklearn.naive_bayes import GaussianNB

In [144]:
gnb = GaussianNB?

In [145]:
gnb = GaussianNB()

In [147]:
gnb.fit(dataToTrain.drop(['label'], axis = 1), dataToTrain['label'])

GaussianNB(priors=None, var_smoothing=1e-09)

In [148]:
preds = gnb.predict(dataToTest.drop(['label'], axis = 1))

In [149]:
preds.sum()

515

In [150]:
dataToTest['label'] = preds
dataToTest.reset_index(inplace = True)
dataToTest['id']=test['id']
dataToTest[['id','label']].head() #.reset_index(inplace = True)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,0
4,7925,1


In [151]:
dataToTest[['id', 'label']].to_csv('out.csv',index = False)

### Word2Vec

In [58]:
import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [59]:
word_vectors['boy'].shape


(300,)

In [202]:
trainTest['cleanTweet_removed_hash'] = trainTest['cleanTweet'].apply(lambda x: x.replace("#", ""))

In [113]:
# features = tfidf.fit_transform(trainTest['cleanTweet'])
# featuresData = pd.DataFrame(features.todense(), columns=tfidf.get_feature_names())

trainTest['cleanTweetW2vEmbed'] = trainTest['cleanTweet_removed_hash'].apply(lambda x: np.mean([word_vectors[word] 
                                                                            for word in x.split() if word in word_vectors ], 
                                                                                  axis = 0))

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [114]:
trainTest.reset_index(drop = True, inplace = True)

In [115]:
len(trainTest['cleanTweetW2vEmbed'])

9873

In [65]:
trainTest["col"+str(2)][3]

KeyError: 'col2'

In [ ]:
import datetime
datetime.datetime.now()

In [110]:
trainTest.shape

(9873, 5)

In [116]:
extract_Embed = np.ones((trainTest.shape[0], 300))

In [117]:
trainTest.columns

Index(['label', 'tweet', 'cleanTweet', 'cleanTweet_removed_hash', 'cleanTweetW2vEmbed'], dtype='object')

In [118]:
trainTest2 = trainTest[['label', 'tweet', 'cleanTweet', 'cleanTweet_removed_hash', 'cleanTweetW2vEmbed']]

In [119]:
trainTest['cleanTweetW2vEmbed'][:5]

0    [-0.077555336, -0.076171875, -0.109436035, 0.09312609, -0.117757164, 0.07720269, 0.1378445, 0.01...
1    [-0.05678489, 0.041581586, -0.0033389004, 0.06356534, -0.07273726, 0.0881958, 0.061601117, -0.12...
2    [-0.029959785, -0.039008245, -0.038450453, 0.15711805, -0.19240993, 0.09033203, 0.051255967, -0....
3    [0.038226787, -0.016042856, 0.032902643, 0.153325, -0.049804688, 0.010714018, 0.0006573017, -0.0...
4    [0.030097269, 0.0167597, 0.020346902, 0.16103293, -0.055641867, 0.047518644, 0.10298295, -0.0509...
Name: cleanTweetW2vEmbed, dtype: object

In [120]:
extract_Embed[:5,:]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [121]:
finalOutput = []

for i in range(trainTest2.shape[0]):
    finalOutput.append(trainTest['cleanTweetW2vEmbed'][i] * extract_Embed[i,:])

In [122]:
cols = ["col"+str(i) for i in range(300)]

finalOutput = pd.DataFrame(data=finalOutput, columns= cols)
finalOutput['label'] = trainTest2['label']

In [81]:
finalOutput.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9873 entries, 0 to 9872
Columns: 301 entries, col0 to label
dtypes: float64(301)
memory usage: 22.7 MB


In [82]:
finalOutput.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205,col206,col207,col208,col209,col210,col211,col212,col213,col214,col215,col216,col217,col218,col219,col220,col221,col222,col223,col224,col225,col226,col227,col228,col229,col230,col231,col232,col233,col234,col235,col236,col237,col238,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255,col256,col257,col258,col259,col260,col261,col262,col263,col264,col265,col266,col267,col268,col269,col270,col271,col272,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282,col283,col284,col285,col286,col287,col288,col289,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299,label
0,-0.077555,-0.076172,-0.109436,0.093126,-0.117757,0.077203,0.137845,0.019763,0.044637,0.091148,-0.079074,-0.106255,0.005398,-0.091621,0.046360,0.135118,0.045932,0.167982,-0.047607,-0.080248,0.004340,-0.026069,0.070855,-0.002959,-0.102403,0.008355,-0.088311,0.124844,0.063182,-0.237684,-0.090237,0.029717,-0.044162,0.031114,-0.033868,-0.083225,-0.094459,0.034153,-0.047350,0.088786,-0.008396,-0.051609,0.154514,0.077610,0.013794,-0.177327,0.036040,0.024292,-0.057617,0.046212,-0.129178,-0.074439,0.003092,0.037120,-0.033617,0.062988,-0.073076,-0.191379,-0.070075,0.050435,0.004801,0.058404,-0.093383,-0.041423,-0.090608,0.111104,-0.128852,0.117099,-0.068454,0.029148,0.085286,0.046265,-0.104333,0.021164,-0.202488,-0.065864,0.026245,0.010449,0.052694,0.075012,-0.046577,0.038235,-0.036879,-0.044271,-0.062185,-0.041477,-0.025011,0.186415,-0.021844,0.010373,-0.018609,-0.074463,-0.157688,-0.045512,0.058065,0.019368,0.119119,-0.065833,0.177599,0.024411,-0.072452,-0.063328,-0.037333,0.011027,0.149089,0.105333,0.010037,-0.008518,0.099141,-0.055800,-0.033474,0.068524,0.112345,-0.012261,0.097236,-0.011963,-0.137044,-0.234490,0.176866,0.028646,-0.022325,-0.114855,-0.201823,0.018148,0.125407,0.054816,-0.082194,0.085097,0.002360,0.113430,-0.130208,-0.046821,-0.128689,0.029195,-0.079739,-0.064467,0.068766,0.039052,0.093981,0.040970,0.080010,-0.094622,-0.087782,-0.062500,-0.095812,0.101562,0.172173,0.089138,-0.119398,-0.150499,0.110162,-0.244168,-0.049099,0.065833,-0.064955,-0.114709,-0.141032,-0.160997,-0.078125,-0.069383,-0.151144,-0.020915,0.038683,-0.001709,-0.085507,-0.213596,0.126085,-0.016113,-0.220839,0.060109,-0.015910,0.103455,-0.055854,-0.142846,-0.133165,0.037699,0.210666,-0.155253,-0.083930,-0.027015,-0.127930,-0.164802,-0.027873,-0.088596,0.059540,0.047106,-0.167372,0.017610,-0.093994,-0.081007,-0.070421,0.056220,-0.140159,-0.075155,-0.003350,0.208110,-0.056925,-0.052463,-0.053033,-0.072645,-0.046285,0.000597,-0.165473,-0.000543,0.019528,0.004890,-0.046122,-0.186537,0.053697,-0.

In [128]:
finalOutput.dropna(subset=['col0'], axis = 0, inplace = True)

In [129]:
finalOutput[finalOutput.col0.isna()]

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205,col206,col207,col208,col209,col210,col211,col212,col213,col214,col215,col216,col217,col218,col219,col220,col221,col222,col223,col224,col225,col226,col227,col228,col229,col230,col231,col232,col233,col234,col235,col236,col237,col238,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255,col256,col257,col258,col259,col260,col261,col262,col263,col264,col265,col266,col267,col268,col269,col270,col271,col272,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282,col283,col284,col285,col286,col287,col288,col289,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299,label


In [190]:
dataToTrain = finalOutput.iloc[:train.shape[0] - 4,:]
dataToTest = finalOutput.iloc[train.shape[0] - 4:,:]

X_train, X_test, Y_train, Y_test = train_test_split(dataToTrain.drop(['label'], axis = 1),
                                                    dataToTrain['label'], test_size = 0.2)



In [191]:
lr = LogisticRegression(penalty = 'l1', C=6, max_iter = 100, class_weight='balanced')
lr.fit(X_train, Y_train)
lr.score(X_test, Y_test)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8737373737373737

In [192]:
print(classification_report(Y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.96      0.87      0.91      1184
         1.0       0.70      0.89      0.78       400

    accuracy                           0.87      1584
   macro avg       0.83      0.88      0.85      1584
weighted avg       0.89      0.87      0.88      1584



In [270]:
trainTest.tail(10)

,label,tweet,cleanTweet,cleanTweet_removed_hash,cleanTweetW2vEmbed,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205,col206,col207,col208,col209,col210,col211,col212,col213,col214,col215,col216,col217,col218,col219,col220,col221,col222,col223,col224,col225,col226,col227,col228,col229,col230,col231,col232,col233,col234,col235,col236,col237,col238,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255,col256,col257,col258,col259,col260,col261,col262,col263,col264,col265,col266,col267,col268,col269,col270,col271,col272,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282,col283,col284,col285,col286,col287,col288,col289,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299
9863,NaN,My Men at the top of Gummers How. #mommyhood #iphone #life #instagram #smile #gmy #photooftheday...,My men top gummer how. #mommyhood #iphon #life #instagram #smile #gmi #photooftheday http://inst...,My men top gummer how. mommyhood iphon life instagram smile gmi photooftheday http://instagr.am/...,"[0.0006385216346153846, 0.04877178485576923, 0.04683743990384615, 0.033590463491586536, 0.011305...",0.024912,0.018036,0.039459,0.143415,-0.022234,0.049212,0.066127,-0.025696,0.093342,0.057714,-0.068445,-0.09714,-0.07135,-0.02778,-0.11819,0.129264,0.071926,0.056536,0.026341,-0.040946,0.024549,0.014539,0.105094,-0.001325,-0.000209,0.038378,-0.066371,0.040436,0.022332,0.009992,-0.037397,0.072283,-0.03751,-0.05406,0.027944,-0.043427,0.088654,0.028756,0.032026,0.03959,0.038954,-0.036067,0.15897,-0.063071,-0.053438,0.00897,-0.051566,-0.05392,-0.05338,0.051147,-0.023558,0.029314,0.000872,-0.001604,-0.006097,0.037436,-0.027701,-0.030151,0.03125,-0.067736,0.023734,0.038947,-0.079729,-0.011274,0.013101,-0.056141,-0.048475,0.087899,-0.072226,0.003401,0.056205,0.015568,0.047619,0.013384,-0.145194,-0.064713,0.063878,0.067344,0.045829,0.109515,-0.038757,0.01008,0.02247,0.014739,-0.072144,-0.062953,-0.055954,0.116438,0.011111,0.009888,0.006485,0.052973,-0.08388,-0.039141,-0.017094,-0.028111,0.030997,0.057844,0.05725,-0.033395,-0.084019,-0.040778,0.050153,0.009678,-0.036922,-0.00565,-0.01998,-0.029234,0.024937,-0.085449,-0.030099,-0.1074,-0.039106,0.022127,0.077802,-0.048477,0.068569,-0.064279,0.045306,0.035557,-0.02496,0.011729,-0.021009,0.107631,-0.02308,-0.056353,-0.066738,0.017194,0.068966,-0.030287,-0.103734,-0.13501,-0.049796,-0.002644,-0.011615,-0.042643,-0.034021,0.072619,-0.010829,0.06885,0.079189,-0.037035,0.020203,-0.028909,0.045166,-0.000536,-0.057077,-0.065744,-0.03329,-0.046221,0.035488,0.047254,-0.1072,-0.033372,-0.022319,0.010132,-0.035017,-0.047991,-0.061733,0.046927,0.03179,0.068976,0.00623,0.002372,-0.

In [331]:
finalOutput.tail()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99,col100,col101,col102,col103,col104,col105,col106,col107,col108,col109,col110,col111,col112,col113,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123,col124,col125,col126,col127,col128,col129,col130,col131,col132,col133,col134,col135,col136,col137,col138,col139,col140,col141,col142,col143,col144,col145,col146,col147,col148,col149,col150,col151,col152,col153,col154,col155,col156,col157,col158,col159,col160,col161,col162,col163,col164,col165,col166,col167,col168,col169,col170,col171,col172,col173,col174,col175,col176,col177,col178,col179,col180,col181,col182,col183,col184,col185,col186,col187,col188,col189,col190,col191,col192,col193,col194,col195,col196,col197,col198,col199,col200,col201,col202,col203,col204,col205,col206,col207,col208,col209,col210,col211,col212,col213,col214,col215,col216,col217,col218,col219,col220,col221,col222,col223,col224,col225,col226,col227,col228,col229,col230,col231,col232,col233,col234,col235,col236,col237,col238,col239,col240,col241,col242,col243,col244,col245,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255,col256,col257,col258,col259,col260,col261,col262,col263,col264,col265,col266,col267,col268,col269,col270,col271,col272,col273,col274,col275,col276,col277,col278,col279,col280,col281,col282,col283,col284,col285,col286,col287,col288,col289,col290,col291,col292,col293,col294,col295,col296,col297,col298,col299,label
9868,-0.001598,0.001998,-0.000601,0.031827,-0.014193,-0.015891,-0.003684,-0.072354,0.027921,0.027610,-0.090110,-0.027521,-0.017745,0.050138,-0.057861,0.058006,0.081942,0.021706,-0.005460,-0.041387,0.078957,0.036665,0.000688,-0.030007,0.057284,0.020674,0.007127,-0.007147,0.030717,-0.024481,-0.007468,-0.003640,-0.046886,-0.063665,-0.028720,0.000223,0.053356,-0.071400,0.014316,0.035733,0.042469,-0.032142,0.069658,-0.005354,-0.019949,-0.083674,-0.028598,0.024181,-0.003041,0.031883,0.075018,-0.018666,0.025302,-0.001820,0.073065,0.011541,0.033453,0.007102,0.015131,-0.043457,-0.043302,0.003085,-0.071622,0.035811,0.025391,-0.015936,-0.010437,0.002000,-0.035156,0.006048,0.010243,-0.012246,0.027699,-0.036330,-0.059470,-0.054546,0.028157,0.004106,0.016968,0.035356,-0.016968,0.022505,0.016481,0.020241,-0.023471,0.011919,-0.038175,0.087269,0.026811,0.051270,0.048062,-0.011941,-0.002919,0.047252,0.039229,-0.034712,-0.020463,0.071555,0.035223,0.010726,-0.016987,-0.036954,0.003106,0.031416,0.063615,-0.082653,0.001687,0.025746,0.031605,0.019295,-0.028720,-0.010454,-0.023149,0.041046,0.105047,-0.045554,-0.033070,-0.013123,0.026545,0.039676,0.003951,0.008983,-0.046387,0.070711,0.032526,-0.040899,-0.044582,-0.046864,0.027743,0.000821,0.003640,0.038796,-0.012021,-0.006184,0.006270,0.032104,0.023005,0.049094,0.015891,0.054599,0.049705,-0.051003,-0.010592,-0.050537,0.044700,0.009655,0.091997,-0.060242,-0.059837,-0.113570,0.047854,-0.001021,0.053506,0.076017,-0.056951,0.002131,0.039329,-0.042769,-0.071866,-0.032826,-0.031616,-0.014890,-0.033385,0.012240,0.055398,-0.034801,0.037195,-0.040527,0.050337,-0.060281,-0.036077,-0.029918,0.007480,-0.037126,-0.056419,-0.050515,0.054399,-0.036044,-0.009699,-0.027499,-0.060480,-0.058599,0.035511,0.013872,-0.009411,-0.093395,-0.033248,-0.012093,0.000555,0.073636,-0.011519,-0.039240,-0.027494,0.005748,-0.013184,-0.015771,0.000605,-0.009011,-0.051891,-0.040328,-0.037542,0.029252,-0.027954,0.024814,-0.029519,-0.014094,0.048517,0.005900,-0.015891,-0.022616,0.

In [74]:
cols = ["col"+str(i) for i in range(300)]

In [225]:
cols

['col0',
 'col1',
 'col2',
 'col3',
 'col4',
 'col5',
 'col6',
 'col7',
 'col8',
 'col9',
 'col10',
 'col11',
 'col12',
 'col13',
 'col14',
 'col15',
 'col16',
 'col17',
 'col18',
 'col19',
 'col20',
 'col21',
 'col22',
 'col23',
 'col24',
 'col25',
 'col26',
 'col27',
 'col28',
 'col29',
 'col30',
 'col31',
 'col32',
 'col33',
 'col34',
 'col35',
 'col36',
 'col37',
 'col38',
 'col39',
 'col40',
 'col41',
 'col42',
 'col43',
 'col44',
 'col45',
 'col46',
 'col47',
 'col48',
 'col49',
 'col50',
 'col51',
 'col52',
 'col53',
 'col54',
 'col55',
 'col56',
 'col57',
 'col58',
 'col59',
 'col60',
 'col61',
 'col62',
 'col63',
 'col64',
 'col65',
 'col66',
 'col67',
 'col68',
 'col69',
 'col70',
 'col71',
 'col72',
 'col73',
 'col74',
 'col75',
 'col76',
 'col77',
 'col78',
 'col79',
 'col80',
 'col81',
 'col82',
 'col83',
 'col84',
 'col85',
 'col86',
 'col87',
 'col88',
 'col89',
 'col90',
 'col91',
 'col92',
 'col93',
 'col94',
 'col95',
 'col96',
 'col97',
 'col98',
 'col99',
 'col100',

In [220]:
trainTest.columns

Index(['label', 'tweet', 'cleanTweet', 'cleanTweet_removed_hash', 'cleanTweetW2vEmbed'], dtype='object')

In [222]:
trainTest['cleanTweetW2vEmbed'].shape

(9873,)

In [219]:
trainTest['tweet']

0    #fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #iger...
1    Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…...
2    We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connec...
3    I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr....
4    What amazing service! Apple won't even talk to me about a question I have unless I pay them $19....
Name: tweet, dtype: object

In [216]:
trainTest['label'].value_counts(dropna=False)

0.0    5894
1.0    2026
NaN    1953
Name: label, dtype: int64

In [218]:
train.shape, test.shape

((7920, 3), (1953, 2))

### Stacking

In [385]:
out1 = pd.read_csv('./over88/454_730751_us_out10.csv')
out2 = pd.read_csv('./over88/454_730751_us_out11.csv')
out3 = pd.read_csv('./over88/454_730751_us_out12_MAJWtIM.csv')
out4 = pd.read_csv('./over88/454_730751_us_out4.csv')
out5 = pd.read_csv('./over88/454_730751_us_out5.csv')
out6 = pd.read_csv('./over88/454_730751_us_out6.csv')
out7 = pd.read_csv('./over88/454_730751_us_out7.csv')
out8 = pd.read_csv('./over88/454_730751_us_out8.csv')
out9 = pd.read_csv('./over88/454_730751_us_out9.csv')
# out10 = pd.read_csv('out10.csv')
# out11 = pd.read_csv('out11.csv')
# out12 = pd.read_csv('out12.csv')
# out1 = pd.read_csv('out2.csv')

In [386]:
out1['avg'] = (out2['label'] + out3['label'] + out4['label'] + out5['label'] + out6['label'] + out7['label'] +
               out1['label'] + out9['label'] + out8['label'] ) / 9

In [387]:
out1['label'] = out1['avg'].apply(lambda x: 1 if x > 0.5 else 0)

In [388]:
out1['label'].sum()

620

In [389]:
out1[['id','label']].to_csv('out13.csv', index = False)

# dataToTest[['id', 'label']].to_csv('out.csv',index = False)

### LSTM, Embedding

In [63]:
trainTest.head()

,id,label,tweet,tweet_len,cleanTweet,cleanTweet_removed_hash
0,1,0.0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #iger...,128,fingerprint pregnanc test httpsgooglhmfqv android app beauti cute health iger iphoneonli iphones...,fingerprint pregnanc test httpsgooglhmfqv android app beauti cute health iger iphoneonli iphones...
1,2,0.0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias…...,131,final transpar silicon case thank uncl yay soni xperia s sonyexperia httpinstagramcompygetjcjm,final transpar silicon case thank uncl yay soni xperia s sonyexperia httpinstagramcompygetjcjm
2,3,0.0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connec...,123,we love would go talk makememori unplug relax iphon smartphon wifi connect httpfbmenlsupcu,we love would go talk makememori unplug relax iphon smartphon wifi connect httpfbmenlsupcu
3,4,0.0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr....,112,im wire i know im georg i made way iphon cute daventri home httpinstagrampliujsk,im wire i know im georg i made way iphon cute daventri home httpinstagrampliujsk
4,5,1.0,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19....,124,what amaz servic appl wont even talk question i unless i pay stupid support,what amaz servic appl wont even talk question i unless i pay stupid support


In [62]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [113]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(trainTest.cleanTweet_removed_hash)

max_len = max([len(x.split()) for x in trainTest.cleanTweet_removed_hash])

vocab_size = len(tokenizer_obj.word_index) + 1

trainTestTokenized = tokenizer_obj.texts_to_sequences(trainTest.cleanTweet_removed_hash)

trainTestTokenizedPadded = pad_sequences(trainTestTokenized, maxlen=max_len, padding="post")

In [75]:
trainTestTokenizedPadded.shape

(9873, 38)

In [99]:
word_index = tokenizer_obj.word_index

In [1]:
import pandas as pd

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [87]:
trainTest.cleanTweetList = trainTest.cleanTweet_removed_hash.apply(lambda x: x.split())

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [88]:
import gensim

model = gensim.models.Word2Vec(sentences=trainTest.cleanTweetList, size = 100, window = 5, workers = 4, min_count = 1)

In [98]:
model.wv.word_vec('good')

array([ 0.6610573 , -0.30802998,  0.24405617, -0.4071091 ,  0.71156204,
        0.46749943,  0.55638534, -1.2319019 , -0.1261279 ,  0.7532104 ,
       -0.10373979,  0.2698762 ,  0.11067977, -0.0490696 , -0.24583685,
       -0.92831665, -0.47254956, -0.30737472,  0.85474366, -0.10203871,
        0.6106124 , -0.3034497 ,  0.02532711, -1.2660023 , -0.35287464,
        0.8538259 , -0.9504076 , -0.3907863 , -0.1418118 ,  0.59802204,
        0.21684664, -0.8220526 , -0.8047163 , -0.31276074,  1.5404094 ,
       -0.33098733, -0.05129599, -0.59069705, -0.49511194,  0.07656218,
        1.5395488 , -1.0093161 , -0.36343956,  0.8229753 , -0.76030743,
        0.05171171, -0.16937049, -0.24778445, -0.2519691 , -0.5372806 ,
        0.03833213,  0.91597426, -0.01075504,  0.5475987 ,  0.7149227 ,
        1.097446  , -0.34020638, -1.0174023 ,  0.4992436 , -0.4638471 ,
        0.36834598, -0.8941949 ,  0.50197697, -0.9468691 ,  0.33906552,
        1.5444349 ,  0.44317314, -0.26164177, -0.3755595 ,  0.56

In [83]:
trainTest.cleanTweet_removed_hash.head()

0    fingerprint pregnanc test httpsgooglhmfqv android app beauti cute health iger iphoneonli iphones...
1         final transpar silicon case thank uncl yay soni xperia s sonyexperia httpinstagramcompygetjcjm
2             we love would go talk makememori unplug relax iphon smartphon wifi connect httpfbmenlsupcu
3                       im wire i know im georg i made way iphon cute daventri home httpinstagrampliujsk
4                            what amaz servic appl wont even talk question i unless i pay stupid support
Name: cleanTweet_removed_hash, dtype: object

In [95]:
model.wv.most_similar('worst')

[('big', 0.9995831251144409),
 ('feel', 0.9993764162063599),
 ('ye', 0.9993605613708496),
 ('year', 0.999345600605011),
 ('load', 0.9993285536766052),
 ('switch', 0.9993219375610352),
 ('onli', 0.9993166327476501),
 ('come', 0.9992817044258118),
 ('run', 0.9992805123329163),
 ('fast', 0.9992787837982178)]

In [100]:
EMBEDDING_DIM = 100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if(i > num_words):
        continue
    embedding_vector = model.wv.word_vec(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [123]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, RNN
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

model = Sequential()
embedding_layer = Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix), input_length=max_len,
                           trainable = False)
model.add(embedding_layer)
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [107]:
trainTestTokenizedPadded[:train.shape[0]].shape[0]

7920

In [124]:
VALIDATION_SPLIT = 0.2

indices = np.arange(trainTestTokenizedPadded[:train.shape[0]].shape[0])
np.random.shuffle(indices)

trainTestTokenizedPadded1 = trainTestTokenizedPadded[indices]
label = trainTest[:train.shape[0]].label[indices]

num_validation_samples = int(VALIDATION_SPLIT * trainTestTokenizedPadded1.shape[0])

X_train_pad = trainTestTokenizedPadded1[:-num_validation_samples]
Y_train = label[:-num_validation_samples]
X_test_pad = trainTestTokenizedPadded1[-num_validation_samples:]
Y_test = label[-num_validation_samples:]

In [125]:
model.fit(X_train_pad, Y_train, batch_size = 128, epochs = 25, validation_data=(X_test_pad, Y_test), verbose=2)

Train on 6336 samples, validate on 1584 samples
Epoch 1/25
 - 5s - loss: 0.6013 - acc: 0.7426 - val_loss: 0.5461 - val_acc: 0.7506
Epoch 2/25
 - 2s - loss: 0.5105 - acc: 0.7500 - val_loss: 0.3924 - val_acc: 0.8258
Epoch 3/25
 - 2s - loss: 0.4228 - acc: 0.7921 - val_loss: 0.3660 - val_acc: 0.8100
Epoch 4/25
 - 2s - loss: 0.3997 - acc: 0.8103 - val_loss: 0.3554 - val_acc: 0.8258
Epoch 5/25
 - 2s - loss: 0.3923 - acc: 0.8101 - val_loss: 0.3554 - val_acc: 0.8213
Epoch 6/25
 - 2s - loss: 0.3794 - acc: 0.8267 - val_loss: 0.3445 - val_acc: 0.8295
Epoch 7/25
 - 2s - loss: 0.3766 - acc: 0.8264 - val_loss: 0.3387 - val_acc: 0.8327
Epoch 8/25
 - 2s - loss: 0.3695 - acc: 0.8303 - val_loss: 0.3352 - val_acc: 0.8396
Epoch 9/25
 - 2s - loss: 0.3564 - acc: 0.8341 - val_loss: 0.3351 - val_acc: 0.8371
Epoch 10/25
 - 2s - loss: 0.3572 - acc: 0.8382 - val_loss: 0.3350 - val_acc: 0.8352
Epoch 11/25
 - 2s - loss: 0.3515 - acc: 0.8384 - val_loss: 0.3268 - val_acc: 0.8453
Epoch 12/25
 - 2s - loss: 0.3533 - ac

In [112]:
trainTestTokenizedPadded[train.shape[0]:]

array([], shape=(0, 38), dtype=int32)

In [126]:
preds = model.predict(trainTestTokenizedPadded[train.shape[0]:])

In [132]:
np.where?

In [133]:
preds = np.where(preds>0.5, 1,0) #preds

In [134]:
preds.sum()

555

In [136]:
dataToTest['label'] = preds
dataToTest.reset_index(inplace = True)
dataToTest['id']=test['id']
dataToTest[['id','label']].head() #.reset_index(inplace = True)
dataToTest[['id', 'label']].to_csv('outLSTM.csv',index = False)

c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\sarthak.agarwal\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
